In [38]:
import pandas as pd
import numpy as np
import datetime
import sys, os
import glob
import time
import fnmatch
import os
import xlwings as xw
from datetime import date
start_time = time.time()

print('Reading EDI File')
a=fnmatch.filter(os.listdir('.'), 'EDI WK*')
a=a[0]
print('Reading', a)

dfEDICOMP = pd.read_excel(a,sheet_name='EDI 862 - DM COMP', dtype={'Supplier_Item_Name': str,'Customer_Item_Name':str})

c=(time.time() - start_time)/60
print("--- %s minutes to read EDI Fle---" % c)
dfEDICOMP = dfEDICOMP[(dfEDICOMP['Site_Name'] == 24260)]#just san mina info
dfEDICOMP = dfEDICOMP[(dfEDICOMP['Data_Measure'] != 'SVServiceDemand')] #delete extra rows
dfEDICOMP['Date'] = pd.to_datetime(dfEDICOMP['Date']).dt.date

#get date from 17 weeks
today = date.today()
monday17 = today + datetime.timedelta(days=-today.weekday(), weeks=16) #monday pf current week plus 16 weeks
#print(monday17)
#create new column to see if row is under 17weeks or not
mask1 = (dfEDICOMP['Date'] <= monday17)
dfEDICOMP['WK'] = np.where(mask1, 'under17','notunder17')
#dfEDICOMP.head()

piv52weeks = pd.crosstab(index=[dfEDICOMP.Customer_Item_Name], columns=[dfEDICOMP["Data_Measure"]], values=dfEDICOMP["Quantity"], aggfunc='sum').fillna(0)
piv52weeks = piv52weeks.drop(['SVHubInventory','SVRMA','SVZOI'], axis = 1)
piv52weeks.reset_index(inplace=True)
#piv52weeks.head()
piv17weeks = dfEDICOMP[(dfEDICOMP['WK'] == 'under17') & (dfEDICOMP['Data_Measure'] == 'SVGrossDemand')]
piv17weeks = pd.crosstab(index=[piv17weeks.Customer_Item_Name], columns=[piv17weeks["Data_Measure"]], values=piv17weeks["Quantity"], aggfunc='sum').fillna(0)
piv17weeks.rename(columns={'SVGrossDemand': 'SVGrossDemand17weeks'},inplace=True)
#piv17weeks.head()
piv52weeks.rename(columns={'SVGrossDemand': 'SVGrossDemand52weeks'},inplace=True)
piv52weeks = pd.merge(piv52weeks, piv17weeks, on='Customer_Item_Name', how='left')
piv52weeks.insert(0, 'Site', '24260')
piv52weeks.rename(columns={'Customer_Item_Name': 'Part_No'},inplace=True)
piv52weeks.insert(2, 'Zebra Part',piv52weeks['Part_No'] )
piv52weeks.insert(3, 'Part Number','')
piv52weeks.insert(4, 'Consumable','')

dfEDIMASTER = pd.read_excel(a, sheet_name='EDI 888 - MASTER DATA', dtype={'ERP_Customer_Item': str,'ZEBRA_Item':str})
dfEDIMASTER.rename(columns={'ERP_Customer_Item': 'Part_No'},inplace=True)

dfEDIMASTER = dfEDIMASTER[(dfEDIMASTER['ERP_Supplier_Code'] == 24260)]#just san mina info
#dfEDIMASTER.loc[dfEDIMASTER['Part_No'] == "56311"]
#dfEDIMASTER.head()

#MERGE EDIMASTER info
piv52weeks = pd.merge(piv52weeks, dfEDIMASTER, on='Part_No', how='left')
#print(len(piv52weeks), 'should be around 5750')

#arrange EDI Master info
mid = piv52weeks['SV_Product_Family']
piv52weeks.drop(labels=['SV_Product_Family'], axis=1, inplace = True)
piv52weeks.insert(5, 'Product Family', mid)
mid = piv52weeks['Product_Family_List']
piv52weeks.drop(labels=['Product_Family_List'], axis=1, inplace = True)
piv52weeks.insert(6, 'Product Family List', mid)
mid = piv52weeks['ERP_Customer_Item_Description']
piv52weeks.drop(labels=['ERP_Customer_Item_Description'], axis=1, inplace = True)
piv52weeks.insert(7, 'Description', mid)
mid = piv52weeks['SV_Tier2_Supplier_Name']
piv52weeks.drop(labels=['SV_Tier2_Supplier_Name'], axis=1, inplace = True)
piv52weeks.insert(8, 'Tier2 Supplier Name', mid)
mid = piv52weeks['ERP_Supplier_Item']
piv52weeks.drop(labels=['ERP_Supplier_Item'], axis=1, inplace = True)
piv52weeks.insert(9, 'Supplier Part', mid) #J Column

piv52weeks.insert(10, 'Receipts1','')
piv52weeks.insert(11, 'Lead Time - Previous IRA','')#l column

mid = piv52weeks['SV_Lead_Time']
piv52weeks.drop(labels=['SV_Lead_Time'], axis=1, inplace = True)
piv52weeks.insert(12, 'Lead Time', mid)

piv52weeks.insert(13, 'Moq- Previous IRA','')#N Column

mid = piv52weeks['SV_MOQ']
piv52weeks.drop(labels=['SV_MOQ'], axis=1, inplace = True)
piv52weeks.insert(14, 'Moq', mid)#O column

piv52weeks.insert(15, 'Raw Cost - Previous IRA', mid)#P column
piv52weeks.insert(16, 'Sanmina cost', '')#Q column
piv52weeks.insert(17, 'Delta', '')#R column


mid = piv52weeks['SV_RAW_Cost']
piv52weeks.drop(labels=['SV_RAW_Cost'], axis=1, inplace = True)
piv52weeks.insert(18, 'Raw Cost', mid)#S column

piv52weeks.insert(19, 'Lblty- Previous IRA', mid)#T column

mid = piv52weeks['SV_LBLTY_Category']
piv52weeks.drop(labels=['SV_LBLTY_Category'], axis=1, inplace = True)
piv52weeks.insert(20, 'Lblty', mid) #U column
piv52weeks.insert(21, 'Ltb Number- Previous IRA', mid)#V column
piv52weeks.insert(22, 'Ltb Number', '')#W column
piv52weeks.insert(23, 'Safety Stock-- Previous IRA', mid)#X column

mid = piv52weeks['SV_Safety_Stock']
piv52weeks.drop(labels=['SV_Safety_Stock'], axis=1, inplace = True)
piv52weeks.insert(24, 'Safety Stock', mid)#Y column
#piv52weeks.head()

#infro from previous IRA File #EDIT
IRA0 = pd.read_excel("SV IRA_01.10.21_REV_B.xlsx",dtype={'Part_No': str},sheet_name='IRA BUY')
piv52weeks = pd.merge(piv52weeks, IRA0, on='Part_No', how='left')
#IRA0.head()

#arrange previous IRA info
piv52weeks['Lead Time - Previous IRA_x']= piv52weeks['Lead Time -  Current IRA']

piv52weeks['Moq- Previous IRA_x']=piv52weeks['Moq - Current IRA']#N Column
piv52weeks['Raw Cost - Previous IRA_x']=piv52weeks['Raw Cost - Current IRA']#P column
piv52weeks['Lblty- Previous IRA_x']=piv52weeks['Lblty - Current IRA']#T column
piv52weeks['Ltb Number- Previous IRA']=piv52weeks['Ltb Number -  Current IRA']#V column
piv52weeks['Safety Stock-- Previous IRA_x']=piv52weeks['Safety Stock - Current IRA']#X column

piv52weeks = piv52weeks.drop(['Zoi',' Soi','Calc Oh Inv','PO Qty','Calc Matl Req 17 Weeks','Calc Matl Req 17 Weeks - Safety Stock',
                             'Calc Matl Req 52 Weeks','Previous Period Total Inventory Cost',
                             'Previous Period Ems Inventory Cost Sanmina Owned',
                             'Previous Period Ira Inventory Cost Zebra Ira Owned','Calc Oh Inv','OH Inv $'], axis = 1)
piv52weeks['New Zebr IRA owned QTY'] = piv52weeks['New Zebr IRA owned QTY'].replace(np.nan, 0)
piv52weeks.insert(25, 'Zoi', piv52weeks['New Zebr IRA owned QTY'])#Z column
piv52weeks.insert(26, 'Hub','')

piv52weeks['New Sanmina Owned QTY'] = piv52weeks['New Sanmina Owned QTY'].replace(np.nan, 0)
#piv52weeks['New Sanmina Owned QTY'].loc[piv52weeks['Part_No'] == "50-12100-1360"]

piv52weeks.insert(27, 'SOI', piv52weeks['New Sanmina Owned QTY'])#AB column
piv52weeks.insert(28, 'Prev Calc Oh Inv', piv52weeks['Zoi'] + piv52weeks['SOI'])#AC column
piv52weeks.insert(29, 'Prev PO Qty', piv52weeks['PO Qty.1'])#AD column
piv52weeks.insert(30, 'Prev Calc Matl Req 17 Weeks', piv52weeks['Calc Matl Req 17 Weeks.1'])#AE column
piv52weeks.insert(31, 'Prev Calc Matl Req 17 Weeks - Safety Stock', piv52weeks['Calc Matl Req 17 Weeks - Safety Stock.1'])#Af column
piv52weeks.insert(32, 'Prev Calc Matl Req 52 Weeks', piv52weeks['Calc Matl Req 52 Weeks.1'])#AG column

piv52weeks.insert(34, 'Previous Period Ems Inventory Cost Sanmina Owned', piv52weeks['SOI']*piv52weeks['Raw Cost - Previous IRA_x'])#AI column
piv52weeks.insert(35, 'Previous Period Ira Inventory Cost Zebra Ira Owned', piv52weeks['Zoi']*piv52weeks['Raw Cost - Previous IRA_x'])#AJ column
#AH =AI+AJ

piv52weeks.insert(33, 'Previous Period Total Inventory Cost', 
                  piv52weeks['Previous Period Ira Inventory Cost Zebra Ira Owned'] + piv52weeks ['Previous Period Ems Inventory Cost Sanmina Owned'])

piv52weeks['Previous Period Total Inventory Cost'].fillna(0)

#AG column
piv52weeks.insert(37,'Calc Oh Inv',piv52weeks['SVSOI'])#AK

piv52weeks.insert(38,'OH Inv $',piv52weeks['Calc Oh Inv']*piv52weeks['Raw Cost'])#AL
piv52weeks.insert(39,'PO Qty',piv52weeks['SVOpenPOQty'])#AM
piv52weeks.insert(40,'Calc Matl Req 17 Weeks',piv52weeks['SVGrossDemand17weeks'])#AN

piv52weeks.insert(41,'Calc Matl Req 17 Weeks - Safety Stock',piv52weeks['Calc Matl Req 17 Weeks']-piv52weeks['Safety Stock'])#AO
piv52weeks.insert(42,'Calc Matl Req 52 Weeks',piv52weeks['SVGrossDemand52weeks'])#AP
piv52weeks=piv52weeks.drop(['SVOpenPOQty','SVGrossDemand52weeks'],axis=1)

finalIRA=piv52weeks.iloc[:,0:42:]
#finalIRA.head()

##San MINA input
print('Reading SanMinaItems')
sanmina1=fnmatch.filter(os.listdir('.'), 'Sanmina items*')
sanmina1=sanmina1[0]
print(sanmina1,' file')
sanmina = pd.read_excel(sanmina1,dtype={'Customer_Item_Name': str})
#sanmina = sanmina[(sanmina['Supplier'] == 24260)]#just san mina info
sanmina = sanmina[["Customer_Item_Name","Make/Buy","Consumable?","Recipts Last Qr"]]
sanmina.rename(columns={'Recipts Last Qr':'Receipts','Customer_Item_Name':'Part_No'},inplace=True)
#sanmina.head()

finalIRA = pd.merge(finalIRA, sanmina, on='Part_No', how='left')
finalIRA['Consumable_x'] = finalIRA['Consumable?']
finalIRA = finalIRA[(finalIRA['Make/Buy'] == 'Buy')]#DROP make items
finalIRA.drop(labels=['Consumable?'], axis=1, inplace = True)
#finalIRA.head()

#AQ
finalIRA['120 days demand plus safety stock']=finalIRA['Safety Stock']+finalIRA['Calc Matl Req 17 Weeks - Safety Stock']
#AR 120 days Excess or underage
finalIRA['120 days Excess or underage']=np.where(finalIRA['Consumable_x']=='Yes',0,
                                        np.where(finalIRA['Calc Matl Req 17 Weeks - Safety Stock']>finalIRA['Calc Oh Inv'],
                                                0,finalIRA['Calc Oh Inv'] - finalIRA['Calc Matl Req 17 Weeks - Safety Stock']))
#AS
finalIRA['New Sanmina Owned QTY'] = np.where(finalIRA['120 days Excess or underage'] == 0,finalIRA['Calc Oh Inv'],
                                             finalIRA['Calc Matl Req 17 Weeks - Safety Stock'])
#AT
finalIRA['New Sanmina Owned Dollar Value'] = finalIRA['New Sanmina Owned QTY'] * finalIRA['Raw Cost - Previous IRA_x']
finalIRA['New Sanmina Owned Dollar Value'] = finalIRA['New Sanmina Owned Dollar Value'].replace(np.nan, 0)
finalIRA['New Sanmina Owned QTY'].fillna(0)
finalIRA['SOI'].fillna(0)
#AU
finalIRA[' New Sanmina Owned QTY change +/- '] = finalIRA['New Sanmina Owned QTY'] - finalIRA['SOI']

#finalIRA.loc[finalIRA['Part_No'] == "50-12100-1360"]

#AV
finalIRA['New Sanmina Owned Dollar change Value +/-'] = finalIRA[' New Sanmina Owned QTY change +/- '] * finalIRA['Raw Cost - Previous IRA_x']
#AW
finalIRA['New Zebr IRA owned QTY']= finalIRA['120 days Excess or underage']
#AX
finalIRA['New Zebr IRA owned Dollar Value']=finalIRA['New Zebr IRA owned QTY'] * finalIRA['Raw Cost - Previous IRA_x']
finalIRA['New Zebr IRA owned Dollar Value']=finalIRA['New Zebr IRA owned Dollar Value'].replace(np.nan, 0)

#AY
finalIRA['New Zebr IRA owned QTY +/-'] = finalIRA['New Zebr IRA owned QTY'] - finalIRA['Zoi']
#AZ
finalIRA['New Zebr IRA owned Dollar Value +/-'] = finalIRA['New Zebr IRA owned QTY +/-'] * finalIRA['Raw Cost - Previous IRA_x']
#BA
finalIRA['Qty according to $$$ paid'] = finalIRA['New Zebr IRA owned Dollar Value +/-'] / finalIRA['Raw Cost - Previous IRA_x']
#BB
finalIRA['New Zebra Reset Qty'] = finalIRA['New Zebr IRA owned QTY']- finalIRA['Zoi']
#BC
finalIRA['$$$ New Zebra Reset '] = finalIRA['New Zebra Reset Qty'] * finalIRA['Raw Cost - Previous IRA_x']
#BD
finalIRA['Adriana total Value'] = ''
#BE
finalIRA['Delta'] =''

#BF
finalIRA['Buyback/ Drawback']=np.where(finalIRA['New Zebr IRA owned Dollar Value +/-'] == 0,'',
                                       np.where(finalIRA['New Zebr IRA owned Dollar Value +/-'] > 0,'Buyback','Drawback'))
#BG
finalIRA['Comments EO- Q4FY2020']=''
#BH 
finalIRA['Pay Y/N']=''
finalIRA['Liability : Denied/Approved']=''#BI 
finalIRA['Approved- Denied Reason']=''#BJ
finalIRA['Where Used']=''#BK

#BL MAke/buy from san mina input, move column
mid = finalIRA['Make/Buy']
finalIRA.drop(labels=['Make/Buy'], axis=1, inplace = True)
finalIRA.insert(64, 'Make/Buy', mid)#O column
#finalIRA.head()

finalIRA['Item Type']='Purchased item'#BM if Buy then Purchased item
#BN Receipts fron SM input, move column
mid = finalIRA['Receipts']
finalIRA.drop(labels=['Receipts'], axis=1, inplace = True)
finalIRA.insert(65, 'Receipts', mid)#O column

finalIRA['Q2FY20 Std Cost $$']=finalIRA['Raw Cost']#BO
#BP
finalIRA['New Zebra IRA STD cost Q4FY19 $$']=finalIRA['Q2FY20 Std Cost $$'] * finalIRA['New Zebr IRA owned QTY']
#BQ
finalIRA['Prepaid(IRA) Reval']=finalIRA['New Zebra IRA STD cost Q4FY19 $$'] - finalIRA['New Zebr IRA owned Dollar Value']
#BR
finalIRA['Sanmina Reval']=finalIRA['Q2FY20 Std Cost $$'] * finalIRA['New Sanmina Owned QTY']
#BS
finalIRA['delta']=finalIRA['Sanmina Reval']-finalIRA['New Sanmina Owned Dollar Value']
finalIRA.head()

strings = time.strftime("%m.%d.%Y")
strings= 'IRA ' + strings 
b=("%s.xlsx" % strings)
finalIRA.to_excel(b, sheet_name= 'IRA',index=False)
#print(len(finalIRA))

c=(time.time() - start_time)/60
#print(c)
#print(a)

print(len(finalIRA), 'number of rows, usually 5750')

Reading EDI File
Reading EDI WK 14 - All Suppliers.xlsx
--- 4.387623735268911 minutes to read EDI Fle---
Reading SanMinaItems
Sanmina items wk14.xlsx  file
5053 number of rows, usually 5750
